<a href="https://colab.research.google.com/github/WhiteChocolate0/ProgramApplication/blob/main/EX04_03_%E5%AE%A2%E6%88%B6%E5%88%86%E7%BE%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EX04-03 客戶分群

```
!wget -O car_models.csv https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv
```


*   將 性別 欄位 Label Encoding
*   將客戶分成 3 群，並解讀各群特質
*   計算 k = 2 ~ 15 的 Calinski-Harbasz Score，找出最佳 k 值




### 資料收集

In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv'
df = pd. read_csv(url)
df.head()

,性別,年齡,收入（千）,消費指數（1~100）
0,女,74,38,81
1,女,51,71,91
2,女,30,65,10
3,女,88,49,17
4,女,55,48,70


### 資料前處理

#### 資料清理

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   性別           200 non-null    object
 1   年齡           200 non-null    int64 
 2   收入（千）        200 non-null    int64 
 3   消費指數（1~100）  200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 6.4+ KB


#### 探索性分析

In [3]:
df_cor = df.drop(columns='性別').corr()
df_cor

,年齡,收入（千）,消費指數（1~100）
年齡,1.000000,0.031519,-0.127454
收入（千）,0.031519,1.000000,0.031476
消費指數（1~100）,-0.127454,0.031476,1.000000


#### 類別轉換

In [5]:
df['性別'].unique ()

array(['女', '男'], dtype=object)

In [6]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder (categories=[['女','男']])
df[['性別']] = encoder.fit_transform(df[['性別']])
df

,性別,年齡,收入（千）,消費指數（1~100）
0,0.0,74,38,81
1,0.0,51,71,91
2,0.0,30,65,10
3,0.0,88,49,17
4,0.0,55,48,70
...,...,...,...,...
195,1.0,86,84,82
196,1.0,59,52,30
197,0.0,63,29,61
198,1.0,67,80,9


### 模型訓練

In [7]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(df)

KMeans(n_clusters=3)

In [10]:
df['cluster'] = kmeans.labels_
df

,性別,年齡,收入（千）,消費指數（1~100）,cluster
0,0.0,74,38,81,2
1,0.0,51,71,91,0
2,0.0,30,65,10,1
3,0.0,88,49,17,1
4,0.0,55,48,70,2
...,...,...,...,...,...
195,1.0,86,84,82,0
196,1.0,59,52,30,1
197,0.0,63,29,61,2
198,1.0,67,80,9,1


### 模型評估

In [12]:
df. groupby('cluster').mean()

,性別,年齡,收入（千）,消費指數（1~100）
cluster,,,,
0,0.636364,49.254545,79.927273,77.945455
1,0.597403,57.428571,61.701299,20.181818
2,0.485294,50.455882,38.500000,68.691176


In [13]:
from sklearn.metrics import calinski_harabasz_score
score = calinski_harabasz_score(df.drop(columns='cluster'), kmeans. labels_)
score

np.float64(97.18095240094486)

### 模型調整

In [14]:
df_nocluster = df.drop(columns='cluster')
for i in range (2,16):
  kmeans = KMeans(n_clusters=i)
  kmeans. fit(df_nocluster)
  score = calinski_harabasz_score(df_nocluster, kmeans.labels_)
  print(f'k={i} score={score}')

k=2 score=111.68583137120146
k=3 score=97.19609370207706
k=4 score=90.54017387973543
k=5 score=77.41825190747612
k=6 score=90.3308345868408
k=7 score=95.17942820485145
k=8 score=88.68996693136762
k=9 score=81.72692025261439
k=10 score=96.09705049555026
k=11 score=92.25281391637577
k=12 score=83.72432595497466
k=13 score=86.97508735746398
k=14 score=79.9185677140843
k=15 score=83.8350922907447
